# Modeling RMBS cash flow

In [83]:

def mortgage_schedule_nn_prepayment(principal, annual_rate, years, payments_per_year=12):
    """
        Returns the cash flow schedule of a mortgage loan

    """
    r = annual_rate / payments_per_year
    n = years * payments_per_year
    payment = principal * r / (1 - (1 + r) ** -n)
    CPR = 0.02
    schedule = []
    balance = principal
    
    for i in range(1, n + 1):
        interest = balance * r
        scheduled_principal = payment - interest
        prepayment = CPR * balance
        total_principal = scheduled_principal + prepayment
        scheduled_balance = get_scheduled_balance(payment, r, n, i, payments_per_year)
        balance -= total_principal
        schedule.append(
            [i, round(payment,3), round(interest,3), round(scheduled_principal,3), 
             round(prepayment,3), round(total_principal,3), 
             round(balance,3) if balance > 0 else 0, round(scheduled_balance,3)])
    
    cash_flow_df = pd.DataFrame(
        schedule, 
        columns=["Period", "Payment", "Interest", "ScheduledPrincipal",  
                 "Prepayment", "TotalPrincipal", "EndingBalance", "ScheduledBalance"], 
        index = np.arange(1, years * payments_per_year + 1, 1))
    
    return cash_flow_df

def get_scheduled_balance(paiement, rate, term, actual_term, period = 12):
    """
    paiement: constant paiement
    rate: annual rate
    term: the maturity of the loan
    actual_term: where we are in the loan life
    period: number of paiements per year.
    """
    return paiement/rate * (1 - (1 + rate)**(-(term-actual_term))) 
    

def SMM(loan_cf):
    pass
    #smm_df = loan_cf[] loan_cf[EndingBalance] / loan_cf[]

In [84]:
mtg = mortgage_schedule_nn_prepayment(200000, 0.045, int(360/12))
print(mtg.head(12))

    Period   Payment  Interest  ScheduledPrincipal  Prepayment  \
1        1  1013.371   750.000             263.371    4000.000   
2        2  1013.371   734.012             279.358    3914.733   
3        3  1013.371   718.285             295.086    3830.851   
4        4  1013.371   702.812             310.558    3748.332   
5        5  1013.371   687.591             325.779    3667.154   
6        6  1013.371   672.618             340.753    3587.296   
7        7  1013.371   657.888             355.483    3508.735   
8        8  1013.371   643.397             369.974    3431.450   
9        9  1013.371   629.142             384.229    3355.422   
10      10  1013.371   615.118             398.253    3280.629   
11      11  1013.371   601.322             412.049    3207.051   
12      12  1013.371   587.750             425.620    3134.669   

    TotalPrincipal  EndingBalance  ScheduledBalance  
1         4263.371     195736.629        199736.629  
2         4194.091     191542.539

In [85]:
print(get_scheduled_balance(1013.37062, 0.045, 360, 1))
print(get_scheduled_balance(1013.37062, 0.045, 360, 2))

22519.344022184967
22519.34388318329


# modeling prepayment

## Single Monthly Mortality Rate SMM

The monthly prepayment rate, or single monthly mortality rate (SMM) measures the percentage of a pool’s principal balance that has prepaid in the current month. It is based on the change in thepool’s factor (survival factor) from one period to the next:

In [31]:
def SMM(cash_flow):
    smm_df = (cash_flow["ScheduledBalance"] - cash_flow["EndingBalance"]) / cash_flow["ScheduledBalance"]
    smm_df.index = cash_flow.index
    return smm_df

In [32]:
print(SMM(mtg).head(10))

1     0.020026
2     0.039754
3     0.059186
4     0.078329
5     0.097187
6     0.115764
7     0.134065
8     0.152094
9     0.169855
10    0.187353
dtype: float64
